## Imports

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torch.utils.data import Subset, DataLoader, Dataset
from torchvision.transforms import Compose, ToTensor, Normalize
import numpy as np
from tqdm import tqdm
import torch.optim as optim
import copy

## Settings

In [5]:
train_fraction = 0.3
n_clients = 5
distribution = "iid"
alpha = 0.1
train_batch_size = 64
test_batch_size = 64
public_batch_size = 64
n_samples_public = 400
n_epochs = 3

## Data loaders

In [6]:
def sample_train_data(train_fraction, transform):
    """ Sample a chosen fraction of the training dataset to use.

        Parameters:
        train_fraction  (float): Fraction of training data to use.
        transform       (torchvision.transforms.Compose): Collection of transforms to apply on data.

        Returns:
        torch.utils.data.Subset: Subset of training data.
    """
    train_data = MNIST(
        root='data', 
        train=True, 
        transform=transform, 
        download=True)
    n_samples = len(train_data.targets)
    index_limit = int(train_fraction * n_samples)
    chosen_indices = np.random.choice(torch.arange(n_samples), size=index_limit, replace=False)
    print(f"\nUsing {index_limit} training samples\n", flush=True)

    return Subset(train_data, chosen_indices)

transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])
train_data = sample_train_data(train_fraction, transform)


Using 18000 training samples



In [7]:
def generate_client_data(n_clients, distribution, alpha):
    """ Generate iid client data or non-iid by sampling from Dirichlet distribution.

        Parameters:
        n_clients       (int): Number of clients.
        distribution    (str): Indicator to sample iid or non-iid.
        alpha           (float): Concentration parameter for Dirichlet distribution.
    """
    labels = np.array([y for (_, y) in train_data])
    n_classes = len(np.unique(labels))
    partition_matrix = np.ones((n_classes, n_clients))

    # iid: Sample from each class until no samples left.
    if distribution == "iid":
        partition_matrix /= n_clients
        local_sets_indices = [np.array([], dtype=int) for _ in range(n_clients)]
        clients_iter = np.arange(n_clients)

        for i in range(n_classes):
            class_indices = np.where(labels == i)[0]

            clients_iter = clients_iter[::-1]
            samples_left = True
            while samples_left:
                for j in clients_iter:
                    if len(class_indices) == 0:
                        samples_left = False
                        break
                    else:
                        sample_idx = np.random.choice(len(class_indices))
                        local_sets_indices[j] = np.append(local_sets_indices[j], class_indices[sample_idx])
                        class_indices = np.delete(class_indices, sample_idx)

    # non-iid: Sample from dirichlet distribution.
    else:
        class_indices = []
        for i in range(n_classes):
            class_indices.append(np.array(range(len(labels)))[labels == i])
        valid_pm = False
        while not valid_pm:
            partition_matrix = np.random.dirichlet((alpha, )*n_clients, n_classes)
            valid_pm = all(np.sum(partition_matrix, axis=0) > 0.01)

        local_sets_indices = [[] for _ in range(n_clients)]
        for each_class in range(n_classes):
            sample_size = len(class_indices[each_class])
            for client in range(n_clients):
                np.random.shuffle(class_indices[each_class])
                local_size = int(np.floor(partition_matrix[each_class, client] * sample_size))
                local_sets_indices[client] += list(class_indices[each_class][:local_size])
                class_indices[each_class] = class_indices[each_class][local_size:]
    
    return local_sets_indices


def get_train_data_loaders(batch_size):
    """ Get list of client training data loaders.

        Parameters:
        n_clients       (int): Number of clients.
        distribution    (str): iid/non-iid distributed data.
        alpha           (float): Concentration parameter for dirichlet distribution.
        batch_size      (int): Batch size for loading training data.

        Returns List[torch.utils.data.DataLoader]
    """
    client_data_loaders = []
    for client_indices in local_sets_indices:
            np.random.shuffle(client_indices)
            client_data_loaders.append(DataLoader(Subset(train_data, client_indices), batch_size))
    return client_data_loaders

local_sets_indices = generate_client_data(n_clients, distribution, alpha)
client_data_loaders = get_train_data_loaders(train_batch_size)

In [8]:
def split_test_public(n_samples):
    labels = np.array([y for (_, y) in test_data])
    n_classes = len(np.unique(labels))
    n_samples_per_class = int(n_samples / n_classes)
    all_indices = np.arange(len(labels))

    public_set_indices = []

    for i in range(n_classes):
        class_indices = np.where(labels == i)[0]
        chosen_indices = list(np.random.choice(class_indices, n_samples_per_class, replace=False))
        public_set_indices.extend(chosen_indices)

    # Fill up to n_samples.
    samples_left = n_samples - len(public_set_indices)
    indices_left = [x for x in all_indices if x not in public_set_indices]
    chosen_indices = list(np.random.choice(indices_left, samples_left, replace=False))
    public_set_indices.extend(chosen_indices)

    test_set_indices = [x for x in all_indices if x not in public_set_indices]

    return Subset(test_data, test_set_indices), Subset(test_data, public_set_indices)

test_data = MNIST(
    root='data', 
    train=False, 
    transform=transform, 
    download=True)

test_data, public_data = split_test_public(n_samples_public)
test_data_loader = DataLoader(test_data, test_batch_size)
public_data_loader = DataLoader(public_data, public_batch_size)

## Models

In [9]:
class Mnist_Cnn(nn.Module):
    def __init__(self):
        super(Mnist_Cnn, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        output = self.fc2(x)
        return output
    
class Mnist_Student(nn.Module):
    def __init__(self):
        super(Mnist_Student, self).__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

## Training clients

In [10]:
local_model = Mnist_Cnn()
loss_function = nn.CrossEntropyLoss()

logits_ensemble = torch.zeros(n_samples_public, 10)
print("Starting training")

for i in range(n_clients):
    model = copy.deepcopy(local_model)
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    for epoch in range(n_epochs):
        train_loss = []
        model.train()
        for x, y in client_data_loaders[i]:
            optimizer.zero_grad()
            output = model(x)
            error = loss_function(output, y)
            error.backward()
            optimizer.step()
            
            train_loss.append(error.item())
            print(f"Client {i+1}/{n_clients}: "
                  f"Epoch {epoch+1}/{n_epochs}: "
                  f"Train loss: {sum(train_loss)/len(train_loss):.3f}", end="\r")
    print("\n")
    
    model.eval()
    logits_local = None
    with torch.no_grad():
        for x, _ in public_data_loader:
            if logits_local is None:
                logits_local = F.softmax(model(x), dim=1)
            else:
                logits_local = torch.cat((logits_local, F.softmax(model(x), dim=1)))
        
    # Increment average
    logits_ensemble = logits_ensemble + (logits_local-logits_ensemble)/(i+1)


Starting training
Client 1/5: Epoch 3/3: Train loss: 0.667

Client 2/5: Epoch 3/3: Train loss: 0.689

Client 3/5: Epoch 3/3: Train loss: 0.711

Client 4/5: Epoch 3/3: Train loss: 0.654

Client 5/5: Epoch 3/3: Train loss: 0.674



## Training student

In [15]:
student_targets = torch.zeros(len(test_data.dataset), 10)
for i in range(n_samples_public):
    idx_public = public_data.indices[i]
    student_targets[idx_public] = logits_ensemble[i]

In [10]:
class StudentData(Dataset):
    def __init__(self, dataset, targets):
        
        self.dataset = dataset
        self.targets = targets
        self.indices = dataset.indices
            
    def __getitem__(self, index):
        if isinstance(index, list):
            data, _ = self.dataset[[i for i in index]]
            target = self.targets[[self.indices[i] for i in index]]
        else:
            data, _ = self.dataset[index]
            target = self.targets[self.indices[index]]
            
        return data, target
        
    
    def __len__(self):
        """Total number of samples"""
        return len(self.indices)

student_data = StudentData(public_data, student_targets)
student_data_loader = DataLoader(student_data, public_batch_size)
data_iter = iter(student_data_loader)
next(data_iter)

[tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           ...,
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],
 
 
         [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           ...,
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
           [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],
 
 
         [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
       

In [16]:
model = Mnist_Student()
loss_function = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

n_epochs_student = 100

model.train()
for epoch in range(n_epochs_student):
    train_loss = []
    for x, y in student_data_loader:
        optimizer.zero_grad()
        output = model(x)
        loss = loss_function(output, y)
        loss.backward()
        optimizer.step()
        
        train_loss.append(loss.item())
    print(f"Epoch {epoch+1}/{n_epochs_student}: "
          f"Train loss: {sum(train_loss)/len(train_loss):.3f}", end="\r")
    

In [17]:
correct = 0
total = 0

with torch.no_grad():
     for x, y in test_data_loader:
            output = model(x)
            for idx, i in enumerate(output):
                if torch.argmax(i) == y[idx]:
                    correct += 1
                total +=1
                
print("Accuracy: ", round(correct/total, 3))

Accuracy:  0.762


In [25]:
alg = "fedavg+feded"
algs = alg.split("+")
ens = ["fede"]

In [27]:
any(alg in algs for alg in ens)

In [28]:
b

False